
<h1 align="center">
  Lava, Dynamic Neural Fields and the Loihi Software Ecosystem

</h1>
<h4 align="center">
    Russell Jarvis post-doctoral researcher at ICNS.
</h4>


<h2 align="center">
<img src="magma_versus_lava.png" 
     width="300" 
     height="350" />
</h2>







<h2 align="center">

    Software Ecosystem and the relationship between Magma and Lava
</h2>

<h1 align="center">
<img src="loihi_diagram.png" 
     width="500" 
     height="650" />
</h1>
     


<h2 align="center">

    Software Ecosystem: Lava is a composable coding paradigm.
</h2>

* Think Plug and Play
* Lava architecture "is inspired from the Communicating Sequential Process (CSP) concept for asynchronous, parallel systems that interact via message passing." -- Qoute from Lava README.md
* "A *Process* can thus be as simple as a single neuron or a synapse, as complex as a full neural network, and as non-neuromorphic as a streaming interface for a peripheral device or an executed instance of regular program code.




### Since Lava is "composable", in theory older Loihi paradigms should be interopable with Lava:
* Intel: NxSDK possibly the same thing as the Loihi Neurocore API
* SNIPS a C code paradigm.

### Direct Qoutes:

* "That the specific components of Magma needed to compile processes specific to Intel Loihi chips remains proprietary to Intel and is not provided through this GitHub site (see below). Similar Magma-layer code for other future commercial neuromorphic platforms likely will also remain proprietary." -- Qoute from Lava README.md
* This is a relevant issue for ICNS, because we would like communicate with our board via an interface with Lava/Loihi respecting conventions.


     
### Lava Dynamic Neural Fields Lava-DNF 
* DNF is the subset of the Lava paradigm that has a lot of useful patterns for specifying biological connectivity. 
* https://github.com/lava-nc/lava-dnf


# Usefulness of Lava for large scale biological modelling work

#### What we want from an interface

- [x] Ability to define LIF Cell populations.
- [x] Means to specify forwards connectivity between layers as populations.
- [x] Means to specify recurrent connectivity whithin a population.
- [x] Inhibitory Synapses (negative weight values possible)
- [x] Capacity to support high cell counts.

#### Not yet in the interface
- [ ] Spike Timing Dependent Plasticity (STDP), or on chip local learning rules (coming).  
- [ ] An ability to specify synaptic delays (hyper-synchronous epileptic network activity).  
- [ ] Ability to visualize the layered architecture (nothing like TorchViz or TensorBoard for ANN architecture yet).
- [ ] Delay Learning (possibly not a planned addition)
- [ ] Adaptive Neurons (supported by SLAYER hard to make interoperable)
- [ ] performance profiling (including power consumption). (coming)



<h1 align="center">
  Below a small code demonstration to show how I established the tick box above a Lava-DNF.
</h1>


To evaluate whether Lava is useful for making bioplausible models, we want to define a variety of overlapping Spiking Neural Network (SNN) architectures, you can think of each one as a weighted directed graph: 




A graph is a function of Vertices and Edges:
<h1 align="center">
 $G(V, E)$
</h1>

Below is a diagram of the Potjan's cortical model. This model can be thought of as the composition of many weighted directed graphs, therefore we will use Lava a supported interface to begin to build a cortical model with the Python Loihi simulator.

<h1 align="center">

<img src="Schematic-diagram-of-the-Potjans-Diesmann-cortical-microcircuit-model.png" 
     width="300" 
     height="350" />

</h1>


## Import relevant modules.

In [1]:
!export JUPYTER_PATH="${JUPYTER_PATH}:$(pwd)/src"


In [2]:

from lava.lib.dnf.operations.operations import Weights
from lava.lib.dnf.operations.operations import *
from lava.proc.lif.process import LIF
from lava.lib.dnf.inputs.rate_code_spike_gen.process import RateCodeSpikeGen
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights
from lava.magma.core.run_configs import Loihi1SimCfg #Loihi simulator, not  Loihi itself.
from lava.magma.core.run_conditions import RunSteps
from lava.proc.monitor.process import Monitor
from lava.proc.monitor.models import PyMonitorModel
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.utils.plotting import raster_plot
from lava.lib.dnf.kernels.kernels import SelectiveKernel

import plotly
import plotly.express as px

from tutorial06_hierarchical_processes import *
import pandas as pd

import numpy as np
import elephant



# Pretend Cortical Model Specs:
## These numbers are nominal. 
* The numbers simply fit the performance of a resource limited laptop
* 2 columns.
* 4 layers
* 1 **E** pop and 1 **I** pop per layer.
* 85 cells per population 170 cells per layer.


### Create layerwise cell populations 

In [3]:

ncolumns=2



In [8]:
# Ex excitatory


ly_2_3_ex = np.ndarray((ncolumns),dtype=object)
ly_4_ex = np.ndarray((ncolumns),dtype=object)
ly_5_ex = np.ndarray((ncolumns),dtype=object)
ly_6_ex = np.ndarray((ncolumns),dtype=object)

# In inhibitory
ly_2_3_in = np.ndarray((ncolumns),dtype=object)
ly_4_in = np.ndarray((ncolumns),dtype=object)
ly_5_in = np.ndarray((ncolumns),dtype=object)
ly_6_in = np.ndarray((ncolumns),dtype=object)


for i in range(0,ncolumns):
    ly_2_3_ex[i] = LIF(shape=(ncells,))
    ly_4_ex[i] = LIF(shape=(ncells,))
    ly_5_ex[i] = LIF(shape=(ncells,))
    ly_6_ex[i] = LIF(shape=(ncells,))


    ly_2_3_in[i] = LIF(shape=(ncells,))
    ly_4_in[i] = LIF(shape=(ncells,))
    ly_5_in[i] = LIF(shape=(ncells,))
    ly_6_in[i] = LIF(shape=(ncells,))

ncells = 85
ncolumns = 2
ly_2_3_ex = LIF(shape=(ncells, ncolumns))
ly_4_ex = LIF(shape=(ncells, ncolumns))
ly_5_ex = LIF(shape=(ncells, ncolumns))
ly_6_ex = LIF(shape=(ncells, ncolumns))


ly_2_3_in = LIF(shape=(ncells, ncolumns))
ly_4_in = LIF(shape=(ncells, ncolumns))
ly_5_in = LIF(shape=(ncells, ncolumns))
ly_6_in = LIF(shape=(ncells, ncolumns))


connections=[]
exc_kernel = SelectiveKernel(amp_exc=0.15,
                         width_exc=ncells/6.0,
                         global_inh=0.0)

#for i in range(0,ncolumns):
    #ly_2_3_ex[i] 2 ly_2_3_ex[i]
conv_kernel = connect(ly_2_3_ex.s_out, ly_2_3_ex.a_in, ops=[Weights(1.0)])
connections.append(conv_kernel)

#ly_4_ex[i] 2 ly_4_ex[i]            
one2onec = connect(ly_4_ex.s_out, ly_4_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)

#ly_5_ex[i] 2 ly_5_ex[i]
one2onec = connect(ly_5_ex.s_out, ly_5_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)

#ly_6_ex[i] 2 ly_6_ex[i]
one2onec = connect(ly_6_ex.s_out, ly_6_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)

conv_kernel = connect(ly_2_3_ex.s_out, ly_2_3_ex.a_in, ops=[Weights(1.0)])
connections.append(conv_kernel)

#ly_4_ex[i] 2 ly_4_ex[i]            
one2onec = connect(ly_4_ex.s_out, ly_4_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)

#ly_5_ex[i] 2 ly_5_ex[i]
one2onec = connect(ly_5_ex.s_out, ly_5_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)

#ly_6_ex[i] 2 ly_6_ex[i]
one2onec = connect(ly_6_ex.s_out, ly_6_ex.a_in, ops=[Weights(1.0)])
connections.append(one2onec)


In [5]:
'''
!pip install bmtk
!pip install sonata

!git clone https://github.com/AllenInstitute/sonata
!git clone https://github.com/AllenInstitute/bmtk/
'''    

'\n!pip install bmtk\n!pip install sonata\n\n!git clone https://github.com/AllenInstitute/sonata\n!git clone https://github.com/AllenInstitute/bmtk/\n'

In [6]:
'''
!cd bmtk/examples/point_450cells/network/
!ls -ltr bmtk/examples/point_450cells/network/
import h5py    
import numpy as np    
f1 = h5py.File("bmtk/examples/point_450cells/network/internal_internal_edges.h5",'r+')  
f1['edges']
'''
#print(dir(f1))
#f1

#import pandas as pd
#df = pd.read_hdf("bmtk/examples/point_450cells/network/internal_internal_edges.h5")
#df
#df = pd.DataFrame(np.array(h5py.File("bmtk/examples/point_450cells/network/internal_internal_edges.h5"))['edges'])
#df
#f1.keys()


'\n!cd bmtk/examples/point_450cells/network/\n!ls -ltr bmtk/examples/point_450cells/network/\nimport h5py    \nimport numpy as np    \nf1 = h5py.File("bmtk/examples/point_450cells/network/internal_internal_edges.h5",\'r+\')  \nf1[\'edges\']\n'

In [7]:
print(dir(ly_2_3_ex))

connections=[]
exc_kernel = SelectiveKernel(amp_exc=0.15,
                         width_exc=ncells/6.0,
                         global_inh=0.0)

for i in range(0,ncolumns):
    #ly_2_3_ex[i] 2 ly_2_3_ex[i]
    conv_kernel = connect(ly_2_3_ex[i].s_out, ly_2_3_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(conv_kernel)
    
    #ly_4_ex[i] 2 ly_4_ex[i]            
    one2onec = connect(ly_4_ex[i].s_out, ly_4_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

    #ly_5_ex[i] 2 ly_5_ex[i]
    one2onec = connect(ly_5_ex[i].s_out, ly_5_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

    #ly_6_ex[i] 2 ly_6_ex[i]
    one2onec = connect(ly_6_ex[i].s_out, ly_6_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)



['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_find_attr_by_type', '_init_proc_member_obj', '_is_compiled', '_model', '_post_init', '_runtime', 'a_in', 'bias', 'bias_exp', 'compile', 'du', 'dv', 'id', 'in_ports', 'init_args', 'is_compiled', 'is_sub_proc_of', 'load', 'name', 'out_ports', 'parent_proc', 'pause', 'proc_params', 'procs', 'ref_ports', 'register_sub_procs', 'run', 'runtime', 's_out', 'save', 'shape', 'stop', 'u', 'v', 'validate_var_aliases', 'var_ports', 'vars', 'vth', 'wait']


TypeError: 'LIF' object is not subscriptable

<h1 align="center"> Create the connectivity pattern </h1>

<h3> 
There are three obvious ways to define connectivity:
    * Kernels
    * One to One
    * Connectivity Matrix.    
</h3>

<h4> 
* First I apply the repeated/stereotyped connections whithin and between layers:
</h4>



<!---

<h1 align="center">
    <img src="Schematic-diagram-of-the-Potjans-Diesmann-cortical-microcircuit-model.png" 
         width="300" 
         height="350" />
</h1>

* from the Potjan's wiring diagram
researchgate.net/figure/Schematic-diagram-of-the-Potjans-Diesmann-cortical-microcircuit-model_fig1_349443713

--->

In [ ]:

connections=[]
exc_kernel = SelectiveKernel(amp_exc=0.15,
                         width_exc=ncells/6.0,
                         global_inh=0.0)

for i in range(0,ncolumns):
    #ly_2_3_ex[i] 2 ly_2_3_ex[i]
    conv_kernel = connect(ly_2_3_ex[i].s_out, ly_2_3_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(conv_kernel)
    
    #ly_4_ex[i] 2 ly_4_ex[i]            
    one2onec = connect(ly_4_ex[i].s_out, ly_4_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

    #ly_5_ex[i] 2 ly_5_ex[i]
    one2onec = connect(ly_5_ex[i].s_out, ly_5_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

    #ly_6_ex[i] 2 ly_6_ex[i]
    one2onec = connect(ly_6_ex[i].s_out, ly_6_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)



## One to one synapse projections between layers

In [ ]:
plot_conn_sankey(conv_kernel,ncells)

In [ ]:

for i in range(0,ncolumns):
    #ly_2_3_in[i] 2 ly_2_3_in[i]
    one2onec = connect(ly_2_3_in[i].s_out, ly_2_3_in[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_4_in[i] 2 ly_4_in[i]
    one2onec = connect(ly_4_in[i].s_out, ly_4_in[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_5_in[i] 2 ly_5_in[i]
    one2onec = connect(ly_5_in[i].s_out, ly_5_in[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_6_in[i] 2 ly_6_in[i]
    one2onec = connect(ly_6_in[i].s_out, ly_6_in[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)

for i in range(0,ncolumns):
    #ly_2_3_ex[i] 2 ly_2_3_in[i]
    one2onec = connect(ly_2_3_ex[i].s_out, ly_2_3_in[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)
    #ly_4_ex[i] 2 ly_4_in[i]
    one2onec = connect(ly_4_ex[i].s_out, ly_4_in[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)
    #ly_5_ex[i] 2 ly_5_in[i]
    one2onec = connect(ly_5_ex[i].s_out, ly_5_in[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)
    #ly_6_ex[i] 2 ly_6_in[i]
    one2onec = connect(ly_6_ex[i].s_out, ly_6_in[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

for i in range(0,ncolumns):
    #ly_2_3_in[i] 2 ly_2_3_exc[i]
    one2onec = connect(ly_2_3_in[i].s_out, ly_2_3_ex[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_4_in[i] 2 ly_4_exc[i]
    one2onec = connect(ly_4_in[i].s_out, ly_4_ex[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_5_in[i] 2 ly_5_exc[i]
    one2onec = connect(ly_5_in[i].s_out, ly_5_ex[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)
    #ly_6_in[i] 2 ly_6_exc[i]
    one2onec = connect(ly_6_in[i].s_out, ly_6_ex[i].a_in, ops=[Weights(-1.0)])
    connections.append(one2onec)

<!--

### Wanted Convergence (green and red) and Divergence (blue)

<h1 align="center">
    <img src="conv_div.png" 
         width="280" 
         height="280"/>
</h1>

## Lava DNF has some interesting divergence patterns

```python
exc_kernel = SelectiveKernel(amp_exc=0.15,
                         width_exc=ncells/6.0,
                         global_inh=0.0)
```                         


* Up to now the network has no convergence and limited divergence patterns between cells.
, so its not so much a network, and more of a series of parallel feedforward lines, like in the figure below:

<h1 align="center">
    <img src="one_to_one_connectivity.png" 
         width="45" 
         height="15"/>
</h1>

### Divergence and some very specific connections between layer connections:
-->
## Smeared Regular Divergence
### Some to some connectivity (not one to one, or one to many)



In [ ]:
for i in range(0,ncolumns):
    one2onec = connect(ly_2_3_ex[i].s_out, ly_5_ex[i].s_out, ops=[Weights(1.0)])
    connections.append(one2onec)

    one2onec = connect(ly_2_3_ex[i].s_out, ly_4_in[i].a_in, ops=[Convolution(exc_kernel)])
    connections.append(one2onec)

    one2onec = connect(ly_2_3_ex[i].s_out, ly_5_in[i].a_in, ops=[Convolution(exc_kernel)])
    connections.append(one2onec)


In [ ]:
fig = px.imshow(connections[-1].weights.init)
fig.show()

In [ ]:
for i in range(0,ncolumns):
    one2onec = connect(ly_5_ex[i].s_out, ly_4_in[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)
    one2onec = connect(ly_6_ex[i].s_out, ly_4_ex[i].a_in, ops=[Convolution(exc_kernel)])
    connections.append(one2onec)
    one2onec = connect(ly_6_ex[i].s_out, ly_2_3_in[i].a_in, ops=[Convolution(exc_kernel)])
    connections.append(one2onec)

# Kernel for divergent inhibition
```python
inh_kernel = MultiPeakKernel(amp_exc=0.0,
                         width_exc=1.0,
                         amp_inh=-0.1,
                         width_inh=ncells/10)
```

In [ ]:
inh_kernel = MultiPeakKernel(amp_exc=0.0,
                         width_exc=1.0,
                         amp_inh=-0.1,
                         width_inh=ncells/10)

for i in range(0,ncolumns):
    one2onec = connect(ly_2_3_ex[i].s_out, ly_2_3_in[i].a_in, ops=[Convolution(inh_kernel)])
    connections.append(one2onec)



In [ ]:
plot_conn_sankey(connections[-1],ncells)

In [ ]:
for i in range(0,ncolumns):
    one2onec = connect(ly_4_ex[i].s_out, ly_2_3_ex[i].s_out, ops=[Weights(1.0)])
    connections.append(one2onec)
    one2onec = connect(ly_4_ex[i].s_out, ly_5_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)
    one2onec = connect(ly_5_ex[i].s_out, ly_6_ex[i].a_in, ops=[Weights(1.0)])
    connections.append(one2onec)

* **Specify** a direct connection between each of the two columns as a demonstration only
layer 2-3 of Column2 (index 1) gets no tonic input, its input is just the output of column 1 (index 0)
ly_2_3_ex[1]


In [ ]:
one2onec = connect(ly_2_3_ex[0].s_out, ly_2_3_ex[1].s_out, ops=[Weights(1.0)])
connections.append(one2onec)

dim=(ncells,ncells)
weights0 = 0.0125*np.random.rand(ncells,ncells)
weights1 = weights0
#instantiate 2 DenseLayers
layer0 = DenseLayer(shape=dim,weights=weights0, bias=4, vth=10)
layer1 = DenseLayer(shape=dim,weights=weights1, bias=4, vth=10)
#connect layer 0 to layer 1
layer0.s_out.connect(layer1.s_in)

many2onec = connect(ly_2_3_in[i].s_out, layer0.s_in, ops=[Weights(0.025)])
one2onec = connect(ly_2_3_ex[0].s_out, ly_2_3_ex[1].s_out, ops=[Weights(1.0)])
connections.append(one2onec)


In [ ]:

dim=(ncells,ncells)
weights0 = 0.0125*np.ones((ncells,ncells))
weights1 = weights0
print(weights0)
#instantiate 2 DenseLayers
layer0 = DenseLayer(shape=dim,weights=weights0, bias=4, vth=10)
layer1 = DenseLayer(shape=dim,weights=weights1, bias=4, vth=10)
#connect layer 0 to layer 1

layer0.s_out.connect(layer1.s_in)
many2onec = connect(ly_2_3_in[i].s_out, layer0.s_in, ops=[Weights(-0.15)])
connections.append(many2onec)

many2onec = connect(ly_2_3_ex[i].s_out, layer0.s_in, ops=[Weights(0.05)])
#print('Layer 1 weights: \n', layer1.weights.get(),'\n')
connections.append(many2onec)

many2onec = connect(ly_2_3_ex[i].s_out, layer0.s_in, ops=[Weights(0.05)])
connections.append(one2onec)
#print('Layer 1 weights: \n', layer1.weights.get(),'\n')


# Divergence and Convergence

In [ ]:
# One to many
many2onec = connect(ly_2_3_in[0].out_ports.members[0] ,layer0.s_in, ops=[Weights(0.05)])
connections.append(one2onec)

In [ ]:
# Many to convergence.
many2onec = connect(ly_2_3_in[0].out_ports.members[0] ,layer0.in_ports.members[0], ops=[Weights(0.05)])
connections.append(one2onec)



# TODO:

- [ ] Use the OSB PyNN model to wire layer to layer connections with established probabilities from Potjan's.
https://github.com/NeuralEnsemble/PyNN/blob/master/examples/Potjans2014/network.py




# Create tonic input for the network.

In [ ]:
spike_generator_1 = RateCodeSpikeGen(shape=ncells,)
center_cell=ncells/2.0
spread_across_number_of_cells=ncells 
gauss_pattern_1 = GaussPattern(shape=ncells,
                               amplitude=80,
                               mean=center_cell,
                               stddev=spread_across_number_of_cells)
gauss_pattern_1.a_out.connect(spike_generator_1.a_in)

for i in range(0,ncolumns):
    _=connect(spike_generator_1.s_out, ly_4_ex[i].a_in, [Weights(1.0)])
    _=connect(spike_generator_1.s_out, ly_4_in[i].a_in, [Weights(0.125)])


# Run the preliminary Potjans model on CPU for 250ms

One awesome property of the Lava paradigm you only have to run a segment of the model to run the whole model.

Model segments seem to have parent child relationships with other segments if they are connected with network connections, and the Loihi compiler seems to understand that.

# Set up the experimental recording rig

In [ ]:
time_steps = 250
monitor_layer0 = Monitor()
monitor_layer0.probe(target=layer0.s_out, num_steps=time_steps)
monitor_ly_2_3_ex = Monitor()
monitor_ly_2_3_ex.probe(target=ly_2_3_ex[0].s_out, num_steps=time_steps)
monitor_ly_2_3_in = Monitor()
monitor_ly_2_3_in.probe(target=ly_2_3_in[0].s_out, num_steps=time_steps)
monitor_ly_4_ex = Monitor()
monitor_ly_4_ex.probe(target=ly_4_ex[0].s_out, num_steps=time_steps)
monitor_input_1 = Monitor()
monitor_input_1.probe(spike_generator_1.s_out, time_steps)
other_column = Monitor()
other_column.probe(ly_2_3_ex[1].s_out, time_steps)
ly_4_ex[0].amplitude = 100
ly_4_ex[0].run(condition=RunSteps(num_steps=time_steps),
        run_cfg=Loihi1SimCfg(select_tag='floating_pt'))

# Get probed data from monitors


In [ ]:

data_input1 = monitor_input_1.get_data()\
    [spike_generator_1.name][spike_generator_1.s_out.name]
data_ly_2_3_ex = monitor_ly_2_3_ex.get_data()\
    [ly_2_3_ex[0].name][ly_2_3_ex[0].s_out.name]
data_ly_2_3_in = monitor_ly_2_3_in.get_data()\
    [ly_2_3_in[0].name][ly_2_3_in[0].s_out.name]
data_ly_4_ex = monitor_ly_4_ex.get_data()\
    [ly_4_ex[0].name][ly_4_ex[0].s_out.name]
data_ly_4_ex = monitor_ly_4_ex.get_data()\
    [ly_4_ex[0].name][ly_4_ex[0].s_out.name]

data_l1=monitor_layer0.get_data()\
[layer0.name][layer0.s_out.name]

#print(data_l1)

## Plot the Spiking Data
### Input Data

In [ ]:
raster_plot(data_input1.T)
cv = compute_cv(data_input1)
print("The coefficient of variation is {0}".format(cv))


In [ ]:
raster_plot(data_l1.T)
cv = compute_cv(data_l1.T)

print("The coefficient of variation is {0}".format(cv))


In [ ]:
raster_plot(data_ly_2_3_ex.T)
cv = compute_cv(data_ly_2_3_ex)
print("The coefficient of variation is {0}".format(cv))

In [ ]:
raster_plot(data_ly_4_ex.T)
cv = compute_cv(data_ly_4_ex)
print("The coefficient of variation is {0}".format(cv))
total = np.hstack([data_ly_2_3_ex.T,data_ly_4_ex.T])
raster_plot(total)


In [ ]:
raster_plot(data_ly_2_3_in.T)
cv = compute_cv(data_ly_2_3_in)
print("The coefficient of variation is {0}".format(cv))


In [ ]:
raster_plot(data_ly_2_3_ex.T)
cv = compute_cv(data_ly_2_3_ex)
print("The coefficient of variation is {0}".format(cv))


* Check to see spiking activity propogated from one column to the other via a dedicated connection

In [ ]:
data_ly_2_3_ex_other = other_column.get_data()\
    [ly_2_3_ex[1].name][ly_2_3_ex[1].s_out.name]
raster_plot(data_ly_2_3_ex_other.T)
cv = compute_cv(data_ly_2_3_ex_other)
print("The coefficient of variation is {0}".format(cv))


### My verdict the overall approach works for some things but not others.



- [x] Fan out achievable with connection kernels.
- [x] Mexican hat weight distributions (lateral inhibition) can be coded.
- [x] Means to specify forwards connectivity between populations.
- [x] Means to specify recurrent connectivity between populations.
- [x] Ability to define LIF Cell populations.
- [x] Inhibitory Synapses (negative weight values possible)
- [x] Capacity to support high cell counts.

    




### Missing


- [ ] Although Lava is advertised as composable (interopable with external code) It is unclear how: 
* arbitrary model of STDP can be translated through the Loihi compiler.
* It's not clear how composable the LAVA approach is with NxSDK and SNIPs 
- [ ] (without breaking Loihi compilation).
- [ ] Loihi compilation is still intentionally opaque (proprietary code).
- [ ] Ability to visualize the whole architecture (nothing like TorchViz for ANN architecture yet).
- [ ] Delay Learning 
- [ ] performance profiling (including power consumption). (coming)
- [ ] No synaptic learning capability yet. 

## Concerns
* Convencience methods for assigning delay distributions to weights.
* Lack of examples of the possibility to to hack in old stuff that already works on Loihi like STDP.
